# Gold Layer - Star Schema Builder

## Overview
This notebook builds analytics-ready star schemas (Gold layer) from Silver tables.

**Star Schemas Created:**
- Conformed Dimensions (DimDate, DimCustomer, DimProduct, DimEmployee)
- Fact Tables optimized for Direct Lake

**Prerequisites:**
- Silver tables created (run 02_transform_to_silver.ipynb first)

**Output:**
- Gold star schema tables ready for Power BI Direct Lake

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from datetime import datetime

print(f"Gold Star Schema Builder Started: {datetime.now()}")

## Configuration

In [ ]:
# Configuration - Naming for Gold layer (no prefix, ready for Power BI)
DIMENSION_MAPPINGS = {
    "DimDate": "DimDate",  # Already in final form from Bronze
    "Silver_DimCustomer": "DimCustomer",
    "Silver_DimProduct": "DimProduct",
    "Silver_DimEmployee": "DimEmployee",
    "DimGeography": "DimGeography"  # Already in final form
}

FACT_MAPPINGS = {
    "Silver_FactSales": "FactSales",
    "Silver_FactSupport": "FactSupport",
    "Silver_FactAttrition": "FactAttrition",
    "Silver_FactInventory": "FactInventory"
}

print(f"Building {len(DIMENSION_MAPPINGS)} dimension tables")
print(f"Building {len(FACT_MAPPINGS)} fact tables")

## Build Conformed Dimensions (Gold)

In [ ]:
print("\n" + "="*80)
print("STEP 1: Building Conformed Dimensions (Gold)")
print("="*80)

for source_table, target_table in DIMENSION_MAPPINGS.items():
    try:
        print(f"\nBuilding {target_table}...")
        
        # Read source (Bronze for DimDate/DimGeography, Silver for others)
        df = spark.table(source_table)
        
        # Select only business columns (exclude metadata)
        metadata_cols = ["_ingestion_timestamp", "_source_file", "row_num"]
        business_cols = [c for c in df.columns if c not in metadata_cols]
        df_clean = df.select(*business_cols)
        
        # Add SCD Type 1 effective date (for future SCD Type 2 implementation)
        if "effective_date" not in df_clean.columns:
            df_clean = df_clean.withColumn("effective_date", current_date())
        
        row_count = df_clean.count()
        
        # Write to Gold layer (overwrite mode for full refresh)
        df_clean.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(target_table)
        
        print(f"✅ {target_table} created: {row_count:,} rows")
        
    except Exception as e:
        print(f"⚠️  Skipping {target_table}: {str(e)}")

print("\n✅ Conformed dimensions created")

## Build Fact Tables (Gold)

In [ ]:
print("\n" + "="*80)
print("STEP 2: Building Fact Tables (Gold)")
print("="*80)

for source_table, target_table in FACT_MAPPINGS.items():
    try:
        print(f"\nBuilding {target_table}...")
        
        # Read Silver table
        df = spark.table(source_table)
        
        # Select only business columns
        metadata_cols = ["_ingestion_timestamp", "_source_file", "row_num"]
        business_cols = [c for c in df.columns if c not in metadata_cols]
        df_clean = df.select(*business_cols)
        
        # Add partition column for performance (date-based)
        if "order_date_id" in df_clean.columns:
            df_clean = df_clean.withColumn("year_month", 
                                           substring(col("order_date_id").cast("string"), 1, 6))
        elif "create_date_id" in df_clean.columns:
            df_clean = df_clean.withColumn("year_month",
                                           substring(col("create_date_id").cast("string"), 1, 6))
        
        row_count = df_clean.count()
        
        # Write to Gold layer with partitioning (if applicable)
        if "year_month" in df_clean.columns:
            df_clean.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .partitionBy("year_month") \
                .saveAsTable(target_table)
        else:
            df_clean.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(target_table)
        
        print(f"✅ {target_table} created: {row_count:,} rows")
        
    except Exception as e:
        print(f"⚠️  Skipping {target_table}: {str(e)}")

print("\n✅ Fact tables created")

## Optimize Delta Tables for Direct Lake

In [ ]:
print("\n" + "="*80)
print("STEP 3: Optimizing Delta Tables for Direct Lake")
print("="*80)

# Get all Gold tables (Dim* and Fact*)
gold_tables = [t.name for t in spark.catalog.listTables() 
               if t.name.startswith("Dim") or t.name.startswith("Fact")]

for table_name in gold_tables:
    try:
        print(f"\nOptimizing {table_name}...")
        
        # Run OPTIMIZE command to compact small files
        spark.sql(f"OPTIMIZE {table_name}")
        
        # Run VACUUM to clean up old files (keep 7 days)
        # Note: In production, adjust retention period as needed
        spark.sql(f"VACUUM {table_name} RETAIN 168 HOURS")
        
        print(f"✅ {table_name} optimized")
        
    except Exception as e:
        print(f"⚠️  Could not optimize {table_name}: {str(e)}")

print("\n✅ Delta table optimization complete")

## Verify Star Schema Relationships

In [ ]:
print("\n" + "="*80)
print("STEP 4: Verifying Star Schema Relationships")
print("="*80)

# Verify FactSales → DimCustomer relationship
try:
    fact_sales = spark.table("FactSales")
    dim_customer = spark.table("DimCustomer")
    
    # Find orphaned customer_ids in FactSales
    orphaned = fact_sales.select("customer_id").distinct() \
        .join(dim_customer.select("customer_id"), "customer_id", "left_anti")
    
    orphan_count = orphaned.count()
    
    if orphan_count == 0:
        print("✅ FactSales → DimCustomer: All customer_ids valid")
    else:
        print(f"⚠️  FactSales → DimCustomer: {orphan_count} orphaned customer_ids")
        print("   Sample orphaned IDs:")
        orphaned.show(5, truncate=False)
except Exception as e:
    print(f"⏭️  Skipping FactSales → DimCustomer check: {str(e)}")

# Verify FactSales → DimProduct relationship
try:
    fact_sales = spark.table("FactSales")
    dim_product = spark.table("DimProduct")
    
    orphaned = fact_sales.select("product_id").distinct() \
        .join(dim_product.select("product_id"), "product_id", "left_anti")
    
    orphan_count = orphaned.count()
    
    if orphan_count == 0:
        print("✅ FactSales → DimProduct: All product_ids valid")
    else:
        print(f"⚠️  FactSales → DimProduct: {orphan_count} orphaned product_ids")
except Exception as e:
    print(f"⏭️  Skipping FactSales → DimProduct check: {str(e)}")

# Verify FactSales → DimDate relationship
try:
    fact_sales = spark.table("FactSales")
    dim_date = spark.table("DimDate")
    
    orphaned = fact_sales.select("order_date_id").distinct() \
        .join(dim_date.select("date_id"), 
              fact_sales.order_date_id == dim_date.date_id, 
              "left_anti")
    
    orphan_count = orphaned.count()
    
    if orphan_count == 0:
        print("✅ FactSales → DimDate: All order_date_ids valid")
    else:
        print(f"⚠️  FactSales → DimDate: {orphan_count} orphaned order_date_ids")
except Exception as e:
    print(f"⏭️  Skipping FactSales → DimDate check: {str(e)}")

print("\n✅ Referential integrity validation complete")

## Generate Star Schema Summary

In [ ]:
print("\n" + "="*80)
print("GOLD LAYER - STAR SCHEMA SUMMARY")
print("="*80)

# List all Gold tables
gold_dims = [t for t in spark.catalog.listTables() if t.name.startswith("Dim")]
gold_facts = [t for t in spark.catalog.listTables() if t.name.startswith("Fact")]

print("\n📊 DIMENSION TABLES")
print("-" * 80)
for table in sorted(gold_dims, key=lambda x: x.name):
    df = spark.table(table.name)
    row_count = df.count()
    col_count = len(df.columns)
    print(f"  {table.name:20s} | {row_count:>10,} rows | {col_count:>3} columns")

print("\n📈 FACT TABLES")
print("-" * 80)
for table in sorted(gold_facts, key=lambda x: x.name):
    df = spark.table(table.name)
    row_count = df.count()
    col_count = len(df.columns)
    
    # Show partitioning info if available
    partition_info = ""
    if "year_month" in df.columns:
        partition_count = df.select("year_month").distinct().count()
        partition_info = f"| {partition_count} partitions"
    
    print(f"  {table.name:20s} | {row_count:>10,} rows | {col_count:>3} columns {partition_info}")

print("\n" + "="*80)
print(f"Completion Time: {datetime.now()}")
print("\n✅ Gold star schema is ready for Power BI Direct Lake!")
print("\nNext Steps:")
print("  1. Create Power BI semantic model with Direct Lake connection")
print("  2. Add relationships in Power BI model view")
print("  3. Import DAX measures from fabric/powerbi/dax-measures.md")
print("  4. Configure Fabric Data Agent")
print("="*80)